# **Homework 1: COVID-19 Cases Prediction (Regression)**

Objectives:
* Solve a regression problem with deep neural networks (DNN).
* Understand basic DNN training tips.
* Familiarize yourself with PyTorch.

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2022-spring@googlegroups.com

# Import packages

In [1]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Some Utility Functions

You do not need to modify this part.

In [2]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, 
                                        [train_set_size, valid_set_size], 
                                        generator=torch.Generator().manual_seed(seed)
                                        )
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

# Dataset

In [3]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Neural Network Model
Try out different model architectures by modifying the class below.

In [4]:
class My_Model(nn.Module):
    def __init__(self):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        self.layers = nn.Sequential(
            nn.LazyLinear(64),
            # nn.Linear(24, 128),
            nn.ReLU(),
            nn.LazyLinear(32),
            nn.ReLU(),
            nn.LazyLinear(1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

# Feature Selection
Choose features you deem useful by modifying the function below.

In [5]:
def select_feat(train_data, valid_data, test_data, feat_idx):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

# Training Loop

In [72]:
class Trainer():
    def __init__(self, train_loader, valid_loader, model, config, device):
        # self.save_hyperparameters(ignore=['ignore'])
        criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.
        
        writer = SummaryWriter() # Writer of tensoboard.
        
        if not os.path.isdir('./models'):
                os.mkdir('./models') # Create directory of saving models.

        n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0
        self.loss_both = math.inf, math.inf, math.inf

        for epoch in range(n_epochs):
            model.train() # Set your model to train mode.
            loss_record = []

            # tqdm is a package to visualize your training progress.
            # train_pbar = tqdm(train_loader, position=0, leave=True)

            # for x, y in train_pbar:
            for x,y in train_loader:
                self.optimizer().zero_grad()               # Set gradient to zero.
                x, y = x.to(device), y.to(device)   # Move your data to device. 
                pred = model(x)             
                loss = criterion(pred, y)
                loss.backward()                     # Compute gradient(backpropagation).
                self.optimizer().step()                    # Update parameters.
                step += 1
                loss_record.append(loss.detach().item())
            
            # print("loss of train = ", loss.detach().item() ) #, end="\r")
                
                # Display current epoch number and loss on tqdm progress bar.
                # train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
                # train_pbar.set_postfix({'loss': loss.detach().item()})
                # train_pbar.refresh() # to show immediately the update

            mean_train_loss = sum(loss_record)/len(loss_record)

            writer.add_scalar('Loss/train', mean_train_loss, step)

            model.eval() # Set your model to evaluation mode.
            loss_record = []
            for x, y in valid_loader:
                x, y = x.to(device), y.to(device)
                with torch.no_grad():
                    pred = model(x)
                    loss = criterion(pred, y)

                loss_record.append(loss.item())
            # print("loss of valid = ", loss.item() ) #,end="\r")
                
            mean_valid_loss = sum(loss_record)/len(loss_record)
            print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}', end='\r')
            
            writer.add_scalar('Loss/valid', mean_valid_loss, step)

            if mean_valid_loss < best_loss:
                best_loss = mean_valid_loss
                torch.save(model.state_dict(), config['save_path']) # Save your best model
                # print('Saving model with loss {:.3f}... at epoch {epoch}'.format(best_loss))
                
                self.loss_both = str(mean_train_loss)[:5], str(mean_valid_loss)[:5], str(best_loss)[:5]
                print(f'Saving model with loss {best_loss:.3f} at epoch {epoch}.' , end='\r')
                # print(f"mean of (train loss, valid loss) = {self.loss_both[0]}, {self.loss_both[1]}.") #, end='\r')
                
                early_stop_count = 0
            else: 
                early_stop_count += 1

            if early_stop_count >= config['early_stop']:
                print(f'Model is not improving, so we halt the training session at epoch {epoch}.')
                print(f'Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}', end='\r')
                return

    def optimizer(self):
        return torch.optim.SGD(
            model.parameters(), 
            lr = 0.001,
            momentum=0.9
            )

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [7]:
class Seperate_DATA():
  def __init__(self, seed, feat_idx, valid_ratio, batch_size):
  # Set seed for reproducibility
    same_seed(seed)

    # train_data size: 2699 x 118 (id + 37 states + 16 features x 5 days) 
    # test_data size: 1078 x 117 (without last day's positive rate)
    train_data, test_data = pd.read_csv('covid.train.csv').values, pd.read_csv('covid.test.csv').values
    train_data, valid_data = train_valid_split(train_data, valid_ratio, seed)

    # Print out the data size.
    print(f"""train_data size: {train_data.shape} 
    valid_data size: {valid_data.shape} 
    test_data size: {test_data.shape}""")

    # Select features
    x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, feat_idx)

    # Print out the number of features.
    print(f'number of features: {x_train.shape[1]}')

    # train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
    #                                             COVID19Dataset(x_valid, y_valid), \
    #                                             COVID19Dataset(x_test)
    
    self.batch_size = batch_size
    self.train_dataset = COVID19Dataset(x_train, y_train)
    self.valid_dataset = COVID19Dataset(x_valid, y_valid)
    self.test_dataset = COVID19Dataset(x_test)

  # Pytorch data loader loads pytorch dataset into batches.
  def train_loader(self):
    return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, pin_memory=True)
  def valid_loader(self):
    return DataLoader(self.valid_dataset, batch_size=self.batch_size, shuffle=True, pin_memory=True)
  def test_loader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, pin_memory=True)


# 調整區

In [8]:
def add_to_class(Class):
    def wrapper(obj):
        setattr(Class, obj.__name__, obj)
    return wrapper

In [39]:
# index_col = 0: 把第一個 column 當成 index
# df0 = pd.read_csv('covid.train.csv', index_col = 0)
df0 = pd.read_csv('covid.train.csv')
corr = df0.corr().iloc[-1]
idx = (abs(corr) > 0.86).tolist()[:-1]
idxx = [ i for i in range(len(idx)) if idx[i] == True ]
len(idxx)

17

In [54]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 666666,      # Your seed number, you can pick your lucky number. :)
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 5000,     # Number of epochs.            
    'batch_size': 64,            
    'early_stop': 600,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': 'models/model.ckpt'  # Your model will be saved here.
}

In [109]:
@add_to_class(My_Model)
def __init__(self):
    super(My_Model, self).__init__()
    self.layers = nn.Sequential(
                # nn.Dropout(0.2),
                nn.LazyBatchNorm1d(),
                nn.LazyLinear(128),
                nn.LeakyReLU(0.1),
                nn.LazyLinear(16),
                # nn.ReLU(),
                # nn.LazyBatchNorm1d(),
                nn.LazyLinear(1)
        )

In [110]:
@add_to_class(Trainer)
def optimizer(self):
  return torch.optim.Adam(
      model.parameters(), 
      lr = 0.001,
      # momentum=0.9
      )

# Start training!

In [111]:
seperate_DATA = Seperate_DATA(config['seed'],idxx , config['valid_ratio'], config['batch_size'])
model = My_Model().to(device) # put your model and data on the same computation device.
trainer = Trainer(seperate_DATA.train_loader(), seperate_DATA.valid_loader(), model, config, device)

train_data size: (2160, 118) 
    valid_data size: (539, 118) 
    test_data size: (1078, 117)
number of features: 17


/Users/chocho/Library/r-miniconda-arm64/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Model is not improving, so we halt the training session at epoch 1512.


In [112]:
config['batch_size'], model.layers, trainer.optimizer(), trainer.loss_both

(64,
 Sequential(
   (0): BatchNorm1d(17, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (1): Linear(in_features=17, out_features=128, bias=True)
   (2): LeakyReLU(negative_slope=0.1)
   (3): Linear(in_features=128, out_features=16, bias=True)
   (4): Linear(in_features=16, out_features=1, bias=True)
 ),
 Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     eps: 1e-08
     foreach: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ),
 ('2.087', '1.082', '1.082'))

# Plot learning curves with `tensorboard` (optional)

`tensorboard` is a tool that allows you to visualize your training progress.

If this block does not display your learning curve, please wait for few minutes, and re-run this block. It might take some time to load your logging information. 

In [113]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

Reusing TensorBoard on port 6006 (pid 23765), started 0:17:48 ago. (Use '!kill 23765' to kill it.)

# Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [114]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model().to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(seperate_DATA.test_loader(), model, device) 
save_pred(preds, 'pred.csv')         

100%|██████████| 17/17 [00:00<00:00, 4858.49it/s]


# Reference
This notebook uses code written by Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)